# 5 Layer Convnet for Classifying 5  Types of Flowers

In [1]:
# import necessary dependencies and files
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from features import Features
from tqdm import tqdm
import os, sys
%matplotlib inline

### Load in the datasets

In [2]:
features = Features()
if not os.path.isfile('datasets.npy'):
    datasets = features.create(flatten=False)
else:
    datasets = np.load('datasets.npy')
X_train, y_train, X_test, y_test = features.train_test_split(datasets)

(50, 50, 3)


In [3]:
datasets[0][0].shape

(50, 50, 3)

In [4]:
print('Length of training set: {:,}'.format(len(y_train)))
print('Length of testing set:  {:,}'.format(len(y_test)))

Length of training set: 135
Length of testing set:  14


In [5]:
print('X_train.shape = {} \t X_test.shape = {}'.format(X_train.shape, X_test.shape))
print('y_train.shape = {} \t y_test.shape = {}'.format(y_train.shape, y_test.shape))

X_train.shape = (135, 50, 50, 3) 	 X_test.shape = (14, 50, 50, 3)
y_train.shape = (135, 10) 	 y_test.shape = (14, 10)


In [6]:
X_train[0].shape

(50, 50, 3)

In [7]:
y_train[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.], dtype=float32)

### Define Hyperparameters

In [8]:
# Image & labels
image_size = features.image_size
image_channel = 3
image_shape = (image_size, image_size, image_channel)
image_shape_flat = image_size * image_size * image_channel
num_classes = len(features.classes)

# Network
filter_size = 5
hidden1_channels = 8
hidden2_channels = 16
hidden3_channels = 32
hidden4_channels = 64
hidden5_channels = 128
fully_connected_1 = 512
fully_connected_2 = 256


# Training
learning_rate = 1e-3
dropout = 0.8
iterations = 0
batch_size = 25

### Helper functions for `weights`, `biases`, `conv2d`, & `max_pool`

In [9]:
# Weight initialization
def weight(shape):
    initial = tf.truncated_normal(shape=shape, stddev=1.0/np.sqrt(shape[-2]))
    return tf.Variable(initial)

# Bias in initialization
def bias(length):
    initial = tf.zeros(shape=[length])
    return tf.Variable(initial)

# Convolutional operation
def conv2d(X, W):
    return tf.nn.conv2d(X, W, strides=[1,1,1,1], padding='SAME')

# Max pooling operation
def max_pool(X):
    return tf.nn.max_pool(X, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# Flatten layer
def flatten(layer):
    layer_shape = layer.get_shape()
    num_features = np.array(layer_shape[1:4], dtype=int).prod()
    layer_flat = tf.reshape(layer, [-1, num_features])
    return layer_flat, num_features

In [10]:
# Placeholder Variables
X = tf.placeholder(tf.float32, [None, image_size, image_size, image_channel], name='X_input')
y = tf.placeholder(tf.float32, [None, num_classes], name='Y_input')
keep_prob = tf.placeholder(tf.float32)
y_true = tf.argmax(y, axis=1)

In [11]:
y

<tf.Tensor 'Y_input:0' shape=(?, 10) dtype=float32>

In [12]:
y_true

<tf.Tensor 'ArgMax:0' shape=(?,) dtype=int64>

## Building the Network
### (1st Convolutional Layer + Max pooling) Input Layer `>` Hidden Layer 1

In [13]:
# X_image = tf.reshape(X, shape=[-1, image_size, image_size, num_channel])

In [14]:
# W_hidden1 = weight(shape=[filter_size, filter_size, image_channel, hidden1_channels])
# b_hidden1 = bias(length=hidden1_channels)
# h_conv1 = tf.nn.relu(conv2d(X, W_hidden1) + b_hidden1)
# h_pool1 = max_pool(h_conv1)

In [15]:
# h_pool1

### (2nd Convolutional Layer + Max pooling) Hidden Layer 1 `>` Hidden Layer 2

In [16]:
# W_hidden2 = weight(shape=[filter_size, filter_size, hidden1_channels, hidden2_channels])
# b_hidden2 = bias(length=hidden2_channels)
# h_conv2 = tf.nn.relu(conv2d(h_pool1, W_hidden2) + b_hidden2)
# h_pool2 = max_pool(h_conv2)

In [17]:
# h_pool2

### (3rd Convolutional Layer + Max pooling) Hidden Layer 2 `>` Hidden Layer 3

In [18]:
# W_hidden3 = weight(shape=[filter_size, filter_size, hidden2_channels, hidden3_channels])
# b_hidden3 = bias(length=hidden3_channels)
# h_conv3 = tf.nn.relu(conv2d(h_pool2, W_hidden3) + b_hidden3)
# h_pool3 = max_pool(h_conv3)

In [19]:
# h_pool3

### (4th Convolutional Layer + Max pooling) Hidden Layer 3 `>` Hidden Layer 4

In [20]:
# W_hidden4 = weight(shape=[filter_size, filter_size, hidden3_channels, hidden4_channels])
# b_hidden4 = bias(length=hidden4_channels)
# h_conv4 = tf.nn.relu(conv2d(h_conv3, W_hidden4) + b_hidden4)
# h_pool4 = max_pool(h_conv4)

In [21]:
# h_pool4

### (5th Convolutional Layer) Hidden Layer 4 `>` Hidden Layer 5

In [22]:
# W_hidden5 = weight(shape=[filter_size, filter_size, hidden4_channels, hidden5_channels])
# b_hidden5 = bias(length=hidden5_channels)
# h_conv5 = tf.nn.relu(conv2d(h_pool4, W_hidden5) + b_hidden5)

In [23]:
# h_conv5

### (1st Fully Connected Layer) Hidden Layer 5 `>` Hidden Layer 6

In [24]:
# h_conv5_flat, num_features = flatten(h_conv5)
# W_fc1 = weight(shape=[num_features, fully_connected_1])
# b_fc1 = bias(length=fully_connected_1)
# h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

In [25]:
# h_fc1

### (2nd Fully Connected Layer + Dropout) Hidden Layer 6 `>` Hidden Layer 7

In [26]:
# W_fc2 = weight(shape=[fully_connected_1, fully_connected_2])
# b_fc2 = bias(length=fully_connected_2)
# h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)
# h_drop = tf.nn.dropout(h_fc2, keep_prob=keep_prob)

In [27]:
# h_drop

### Readout/Output Layer

In [28]:
# W_out = weight(shape=[fully_connected_2, num_classes])
# b_out = bias(length=num_classes)
# y_pred = tf.matmul(h_drop, W_out) + b_out
# y_pred_true = tf.argmax(y_pred, axis=1)

In [29]:
# y_pred

In [30]:
# y_pred_true

## Function for constructing the convnet

In [31]:
def convnet(train=False):
    with tf.name_scope('conv_layer1'):
        W_hidden1 = weight(shape=[filter_size, filter_size, image_channel, hidden1_channels])
        b_hidden1 = bias(length=hidden1_channels)
        h_conv1 = tf.nn.relu(conv2d(X, W_hidden1) + b_hidden1)
        h_pool1 = max_pool(h_conv1)
    with tf.name_scope('conv_layer2'):
        W_hidden2 = weight(shape=[filter_size, filter_size, hidden1_channels, hidden2_channels])
        b_hidden2 = bias(length=hidden2_channels)
        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_hidden2) + b_hidden2)
        h_pool2 = max_pool(h_conv2)
    with tf.name_scope('conv_layer3'):
        W_hidden3 = weight(shape=[filter_size, filter_size, hidden2_channels, hidden3_channels])
        b_hidden3 = bias(length=hidden3_channels)
        h_conv3 = tf.nn.relu(conv2d(h_pool2, W_hidden3) + b_hidden3)
        h_pool3 = max_pool(h_conv3)
    with tf.name_scope('conv_layer4'):
        W_hidden4 = weight(shape=[filter_size, filter_size, hidden3_channels, hidden4_channels])
        b_hidden4 = bias(length=hidden4_channels)
        h_conv4 = tf.nn.relu(conv2d(h_conv3, W_hidden4) + b_hidden4)
        h_pool4 = max_pool(h_conv4)
    with tf.name_scope('conv_layer5'):
        W_hidden5 = weight(shape=[filter_size, filter_size, hidden4_channels, hidden5_channels])
        b_hidden5 = bias(length=hidden5_channels)
        h_conv5 = tf.nn.relu(conv2d(h_pool4, W_hidden5) + b_hidden5)
        h_conv5_flat, num_features = flatten(h_conv5)
    with tf.name_scope('fully_connected1'):
        W_fc1 = weight(shape=[num_features, fully_connected_1])
        b_fc1 = bias(length=fully_connected_1)
        h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)
        if train:
            h_fc1 = tf.nn.dropout(h_fc1, keep_prob=keep_prob)
    with tf.name_scope('fully_connected2'):
        W_fc2 = weight(shape=[fully_connected_1, fully_connected_2])
        b_fc2 = bias(length=fully_connected_2)
        h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)
        if train:
            h_drop = tf.nn.dropout(h_fc2, keep_prob=keep_prob)
    with tf.name_scope('output_layer'):
        W_out = weight(shape=[fully_connected_2, num_classes])
        b_out = bias(length=num_classes)
        y_pred = tf.matmul(h_drop, W_out) + b_out
        y_pred_norm = tf.nn.softmax(y_pred)
        y_pred_true = tf.argmax(y_pred_norm, axis=1)
    return y_pred, y_pred_norm, y_pred_true

# Run the convnet
y_pred, y_pred_norm, y_pred_true = convnet(train=True)

### Cost function, and optimizer

In [32]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y, name='xentropy')
cost = tf.reduce_mean(cross_entropy, name='xentropy_mean')
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

### Evaluating accuracy

In [33]:
correct = tf.equal(y_true, y_pred_true)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

## Running tensorflow's `Session()`

In [34]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

### `optimize()` and `print_accuracy()` function helper

In [35]:
# feed_dict_train = {X: X_train, 
#                    y: y_train, 
#                    keep_prob:dropout}
feed_dict_test = {X: X_test, 
                  y: y_test, 
                  keep_prob: dropout}


# Optimize helper
def optimize(num_iter=1):
    global iterations
    for i in tqdm(range(num_iter)):
#         start = 0
#         while start < len(X_train):
#             end = start + batch_size
#             batch_X = X_train[start:end]
#             batch_y = y_train[start:end]
        sess.run(optimizer, feed_dict={X:X_train,
                                       y:y_train,
                                       keep_prob:dropout})
#             start += batch_size
        iterations += 1


# Accuracy helper
def print_accuracy():
    acc = sess.run(accuracy, feed_dict=feed_dict_test)
    print('Accuracy after {:,} iterations = {:.2%}'.format(iterations, acc))


# Predict
def predict(img_path):
    img = feature.preprocess(img_path)
    _, y_pred_norm, y_pred_true = convnet()
    _prob, _pred = sess.run([y_pred_norm, y_pred_true], feed_dict={X: img})

## Training the network

In [36]:
print_accuracy()

Accuracy after 0 iterations = 0.00%


In [37]:
optimize() # for 1 iteration
print_accuracy()

100%|██████████| 1/1 [00:01<00:00,  1.15s/it]

Accuracy after 1 iterations = 7.14%


In [38]:
optimize(9) # to complete 10 iterations
print_accuracy()

100%|██████████| 9/9 [00:09<00:00,  1.05s/it]

Accuracy after 10 iterations = 28.57%


In [ ]:
optimize(90) # to complete 100 iterations
print_accuracy()

100%|██████████| 90/90 [01:41<00:00,  1.20s/it]

Accuracy after 100 iterations = 35.71%


In [ ]:
optimize(900) # to complete 1,000 iterations
print_accuracy()

 24%|██▎       | 212/900 [03:47<13:23,  1.17s/it]

In [ ]:
# optimize(9000) # to complete 10,000 iterations
# print_accuracy()

### Closing the tesorflow's `Session()`

In [ ]:
# sess.close()